First af all we are going to include and import all modules we need for our analysis. 

In [1]:
import boto3
import pandas as pd
from dotenv import load_dotenv
import os
from io import BytesIO


We have our credentials stored in a .env file that has been also included in the gitignore file for privacy purposes

In [5]:
load_dotenv("credentials.env")

aws_access_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_region = os.getenv('AWS_DEFAULT_REGION')

This part is for initialating our access to S3 via boto3 client. 

In [3]:
# Initialize S3 client
s3 = boto3.client(
    's3',
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    region_name=os.getenv('AWS_DEFAULT_REGION')
)

# Define the bucket and prefix
bucket_name = 'zrive-ds-data'
prefix = 'groceries/sampled-datasets/'

# List objects in the S3 bucket
response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

# Print the list of files
if 'Contents' in response:
    for obj in response['Contents']:
        print(obj['Key'])
else:
    print("No files found in the bucket.")


groceries/sampled-datasets/abandoned_carts.parquet
groceries/sampled-datasets/inventory.parquet
groceries/sampled-datasets/orders.parquet
groceries/sampled-datasets/regulars.parquet
groceries/sampled-datasets/users.parquet


Now we are importing our datasets from parquet to Pandas for betteer handling of the information. 

In [6]:
# Define the file you want to download (e.g., orders.parquet)

orders = 'groceries/sampled-datasets/orders.parquet'
regulars ='groceries/sampled-datasets/regulars.parquet'
inventory = 'groceries/sampled-datasets/inventory.parquet'
abandoned_carts = 'groceries/sampled-datasets/abandoned_carts.parquet'
users= 'groceries/sampled-datasets/users.parquet'

# Download the Parquet file from S3

response_orders = s3.get_object(Bucket=bucket_name, Key=orders)
response_regulars = s3.get_object(Bucket=bucket_name, Key=regulars)
response_inventory = s3.get_object(Bucket=bucket_name, Key=inventory)
response_abandoned_carts = s3.get_object(Bucket=bucket_name, Key=abandoned_carts)
response_users = s3.get_object(Bucket=bucket_name, Key=users)

# Load the Parquet file into a Pandas DataFrame
df_orders = pd.read_parquet(BytesIO(response_orders['Body'].read()))
df_regulars = pd.read_parquet(BytesIO(response_regulars['Body'].read()))
df_inventory = pd.read_parquet(BytesIO(response_inventory['Body'].read()))
df_abandoned_carts = pd.read_parquet(BytesIO(response_abandoned_carts['Body'].read()))
df_users = pd.read_parquet(BytesIO(response_users['Body'].read()))



Now we Can start studying our datasets. First of all we want to understand what variables we have in each of our data-sets and get the first conclusions. 


In [8]:
df_orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8773 entries, 10 to 64538
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   id              8773 non-null   int64         
 1   user_id         8773 non-null   object        
 2   created_at      8773 non-null   datetime64[us]
 3   order_date      8773 non-null   datetime64[us]
 4   user_order_seq  8773 non-null   int64         
 5   ordered_items   8773 non-null   object        
dtypes: datetime64[us](2), int64(2), object(2)
memory usage: 479.8+ KB


First conclusion we can draw is that there are no-nulls in our oders

In [10]:
df_regulars.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18105 entries, 3 to 37720
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   user_id     18105 non-null  object        
 1   variant_id  18105 non-null  int64         
 2   created_at  18105 non-null  datetime64[us]
dtypes: datetime64[us](1), int64(1), object(1)
memory usage: 565.8+ KB
